# Tutorial: Generating Structured Output with Loop-Based Auto-Correction

- **Level**: Intermediate
- **Time to complete**: 15 minutes
- **Prerequisites**: You must have an API key from an active OpenAI account as this tutorial is using the gpt-3.5-turbo model by OpenAI.
- **Components Used**: `PromptBuilder`, `GPTGenerator`, `OutputParser` (Custom component)
- **Goal**: After completing this tutorial, you will have built a system that extracts unstructured data, puts it in a JSON schema, and automatically corrects errors in the JSON output from a large language model (LLM) to make sure it follows the specified structure.

> This tutorial uses Haystack 2.0 Beta. To learn more, see [Haystack 2.0 Documentation](https://docs.haystack.deepset.ai/v2.0/docs).

## Overview
This tutorial demonstrates how to use Haystack 2.0's advanced [looping pipelines](https://docs.haystack.deepset.ai/v2.0/docs/pipelines#loops) with LLMs for more dynamic and flexible data processing. You'll learn how to extract structured data from unstructured data using an LLM, and to validate the generated output against a predefined schema.

This tutorial uses gpt-3.5-turbo to change unstructured passages into JSON outputs that follow the [Pydantic](https://github.com/pydantic/pydantic) schema. It uses a custom OutputParser component to validate the JSON and loop back to make corrections, if necessary.

## Preparing the Colab Environment

Enable the debug mode of logging:

In [ ]:
import logging

logging.basicConfig()
logging.getLogger("canals.pipeline.pipeline").setLevel(logging.DEBUG)

## Installing Dependencies
Install Haystack 2.0 Beta and [colorama](https://pypi.org/project/colorama/) with pip:

In [ ]:
%%bash

pip install haystack-ai
pip install colorama

### Enabling Telemetry

Enable telemetry to let us know you're using this tutorial. (You can always opt out by commenting out this line). For details, see [Telemetry](https://docs.haystack.deepset.ai/docs/telemetry).

In [ ]:
from haystack.telemetry import tutorial_running

tutorial_running(29)

## Creating a Custom Component: OutputParser

OutputParser is a custom component that validates if the JSON the LLM generates complies with the pydantic schema. If it doesn't, OutputParser sends back an error message along with the incorrect JSON object to get it fixed in the next loop.

For more details, see [Creating Custom Components](https://docs.haystack.deepset.ai/v2.0/docs/custom-components).

In [ ]:
import json
import random
import pydantic
from pydantic import ValidationError
from typing import Optional, List
from colorama import Fore
from haystack import component

# Define the component input parameters
@component
class OutputParser:
    def __init__(self, pydantic_model: pydantic.BaseModel):
        self.pydantic_model = pydantic_model
        self.iteration_counter = 0

    # Define the component output
    @component.output_types(valid_replies=List[str], invalid_replies=Optional[List[str]], error_message=Optional[str])
    def run(self, replies: List[str]):

        self.iteration_counter += 1

        ##### CREATE AN INVALID REPLY (FOR DEMO PURPOSES) #####
        # Simulate a corrupt JSON randomly by adding extra brackets
        # In a working application, you won't need this section
        if random.randint(0, 100) < 30:
            replies[0] = "{{" + replies[0]
        #######################################################

        try:
            output_dict = json.loads(replies[0])
            self.pydantic_model.parse_obj(output_dict)
            print(
                Fore.GREEN
                + f"OutputParser at Iteration {self.iteration_counter}: Valid JSON from LLM - No need for looping: {replies[0]}"
            )
            return {"valid_replies": replies}

        except (ValueError, ValidationError) as e:
            print(
                Fore.RED
                + f"OutputParser at Iteration {self.iteration_counter}: Invalid JSON from LLM - Let's try again.\n"
                f"Output from LLM:\n {replies[0]} \n"
                f"Error from OutputParser: {e}"
            )
            return {"invalid_replies": replies, "error_message": str(e)}

## Defining a Schema to Parse the JSON Object

Define a simple schema for the data you want to extract from a text passsage using the LLM. The output of the LLM should always be compliant with this `json_schema`.


In [ ]:
from pydantic import BaseModel


class City(BaseModel):
    name: str
    country: str
    population: int


class CitiesData(BaseModel):
    cities: List[City]


json_schema = CitiesData.schema_json(indent=2)
output_parser = OutputParser(pydantic_model=CitiesData)

## Creating the Prompt

Write instructions for the LLM for converting a passage into a JSON format. Ensure the instructions explain how to identify and correct errors if the JSON doesn't match the required schema. Once you create the prompt, initialize PromptBuilder to use it.  

For information about Jinja2 template and PromptBuilder, see [PromptBuilder](https://docs.haystack.deepset.ai/v2.0/docs/promptbuilder).

In [ ]:
from haystack.components.builders import PromptBuilder

prompt_template = """
Create a JSON object from the information present in this passage: {{passage}}.
Only use information that is present in the passage. Follow this JSON schema, but only return the actual instances without any additional schema definition:
{{schema}}
Make sure your response is a dict and not a list.
{% if invalid_replies and error_message %}
  You already created the following output in a previous attempt: {{invalid_replies}}
  However, this doesn't comply with the format requirements from above and triggered this Python exception: {{error_message}}
  Correct the output and try again. Just return the corrected output without any extra explanations.
{% endif %}
"""
prompt_builder = PromptBuilder(template=prompt_template)

## Initalizing the Generator

[GPTGenerator](https://docs.haystack.deepset.ai/v2.0/docs/gptgenerator) generates text using OpenAI's gpt-3.5-turbo model by default. Provide an API key and a model name to the Generator.

In [ ]:
import os
from getpass import getpass

from haystack.components.generators import GPTGenerator

llm_api_key = os.getenv("OPENAI_API_KEY", getpass("Enter OpenAI API key:"))
generator = GPTGenerator(api_key=llm_api_key)

## Building the Pipeline

Add all components to your pipeline and connect them. Add connections from `output_parser` back to the `prompt_builder` for cases where the produced JSON doesn't comply with the JSON schema. Set `max_loops_allowed` to avoid infinite looping.

In [ ]:
from haystack import Pipeline

pipeline = Pipeline(max_loops_allowed=5)

# Add components to your pipeline
pipeline.add_component(instance=prompt_builder, name="prompt_builder")
pipeline.add_component(instance=generator, name="llm")
pipeline.add_component(instance=output_parser, name="output_parser")

# Now, connect the components to each other
pipeline.connect("prompt_builder", "llm")
pipeline.connect("llm", "output_parser")
# If a component has more than one output or input, explicitly specify the connections:
pipeline.connect("output_parser.invalid_replies", "prompt_builder.invalid_replies")
pipeline.connect("output_parser.error_message", "prompt_builder.error_message")

### Visualize the Pipeline

Draw the pipeline with the `draw()` method to confirm the connections are correct. You can find the diagram in the Files section of this Collab.

In [ ]:
pipeline.draw("auto-correct-pipeline.png")

## Testing the Pipeline

Run the pipeline with an example passage that you want to convert into a JSON format and the correct `json_schema`.

> If you encounter `PipelineMaxLoops: Maximum loops count (5) exceeded for component 'prompt_builder'.` error, run the pipeline again. You're getting this error because `OutputParser` randomly corrupts the JSON, so it may happen that 5 loops are not enough to get the correct output.

In [ ]:
passage = "Berlin is the capital of Germany. It has a population of 3,850,809"
result = pipeline.run({"prompt_builder": {"passage": passage, "schema": json_schema}})

print(result)

### Print the Correct JSON
If you didn't get any error, you can now print the corrected JSON.

In [ ]:
valid_reply = result["output_parser"]["valid_replies"][0]
valid_json = json.loads(valid_reply)
print(valid_json)

🎉 Congratulations! You've built a system that generates structured JSON out of unstructured text passages, and auto-corrects it by using the looping functionality of Haystack pipelines.